In [3]:
from lib.polar_histogram import PolarHistogram
from lib.histogram_grid import HistogramGrid
from lib.path_planner import PathPlanner

num_bins = 12 # each bin is 36 degrees
active_region_dimension = (30, 30)
resolution = 1 # 1cm per node
map_fname = 'map.txt'

polar_histogram = PolarHistogram(num_bins)
histogram_grid = HistogramGrid.from_txt(map_fname, active_region_dimension, resolution)
path_planner = PathPlanner()
path_planner = PathPlanner(polar_histogram, histogram_grid)
# print(path_planner)
print(path_planner.get_best_angle())

TypeError: __init__() missing 2 required positional arguments: 'polar_histogram' and 'histogram_grid'

In [ ]:
# from lib.histogram_grid import HistogramGrid

# map_fname = 'map.txt'
# resolution = 1 # node size = 1cm
# hg = HistogramGrid.build_histogram_from_txt(map_fname, resolution)
# print(*hg.histogram_grid, sep="\n")